In [ ]:
%matplotlib inline
from pylab import *

In [ ]:
from deficit_defs import *
from tqdm.notebook import tqdm

Splikes version  0.1.4
Plasticnet version  0.1.12
macOS-14.4.1-arm64-arm-64bit
Base Image File: asdf/bbsk081604_all_log2dog.asdf


In [ ]:
import ray 
ray.init(num_cpus=4)

2024-05-11 11:17:23,307	INFO worker.py:1749 -- Started a local Ray instance.


Python version:,3.11.8
Ray version:,2.20.0


## From a deficit, compare the rates of 

- fix (just lenses)
- patch
- blur
- contrast
- constrast + mask

Fixed params

- open eye noise =0.1
- time of 100 hours (arbitrary)

Measure

- response (max response to gratings, across spatial freq and orientiation)
- recovery rate per hour

In [ ]:
@ray.remote
def run_one_continuous_fix(params,run=True,overwrite=False):
    import plasticnet as pn
    count,eta,noise,number_of_neurons,sfname=(params.count,params.eta,params.noise,
                                        params.number_of_neurons,params.sfname)
    
    if not overwrite and os.path.exists(sfname):
        return sfname
    
    seq=pn.Sequence()
    # deliberately use a standard deficit, with it's own eta and noise
    seq+=deficit(number_of_neurons=params.number_of_neurons) 

    seq+=fix(total_time=100*hour,
             save_interval=20*minute,number_of_neurons=params.number_of_neurons,
             eta=eta,noise=noise)

    if run:
        seq.run(display_hash=False)
        pn.save(sfname,seq) 
    
    return sfname
    

use $\eta=1e-6$ for everything to make the comparison clear

In [ ]:
@ray.remote
def run_one_continuous_blur(params,run=True,overwrite=False):
    import plasticnet as pn
    count,blur,eta,noise,number_of_neurons,sfname=(params.count,params.blur,params.eta,params.noise,
                                        params.number_of_neurons,params.sfname)
    
    if not overwrite and os.path.exists(sfname):
        return sfname
    
    
    seq=pn.Sequence()
    # deliberately use a standard deficit, with it's own eta and noise
    seq+=deficit(number_of_neurons=params.number_of_neurons) 

    seq+=treatment(blur=blur,
                   noise=0.1,
                   noise2=noise,  # treated (strong-eye) noise
                   total_time=100*hour,number_of_neurons=params.number_of_neurons,
                   eta=eta,
                   save_interval=20*minute)
    

    if run:
        seq.run(display_hash=False)
        pn.save(sfname,seq) 
    
    return sfname
    
    
@ray.remote
def run_one_continuous_patch(params,run=True,overwrite=False):
    import plasticnet as pn
    count,eta,noise,number_of_neurons,sfname=(params.count,params.eta,params.noise,
                                        params.number_of_neurons,params.sfname)
    
    if not overwrite and os.path.exists(sfname):
        return sfname
    
    
    seq=pn.Sequence()
    # deliberately use a standard deficit, with it's own eta and noise
    seq+=deficit(number_of_neurons=params.number_of_neurons) 
    
    seq+=patch_treatment(patch_noise=noise,
               total_time=100*hour,number_of_neurons=params.number_of_neurons,
               eta=eta,
               save_interval=20*minute)

    if run:
        seq.run(display_hash=False,print_time=True)
        pn.save(sfname,seq) 
    
    return sfname
        

In [ ]:
@ray.remote
def run_one_continuous_mask(params,run=True,overwrite=False):
    import plasticnet as pn
    count,eta,contrast,mask,f,number_of_neurons,sfname=(params.count,params.eta,params.contrast,params.mask,params.f,
                                        params.number_of_neurons,params.sfname)
    
    if not overwrite and os.path.exists(sfname):
        return sfname

    
    seq=pn.Sequence()
    # deliberately use a standard deficit, with it's own eta and noise
    seq+=deficit(number_of_neurons=params.number_of_neurons) 

    seq+=treatment(f=f,
                   mask=mask,
                   contrast=contrast,
                   total_time=100*hour,
                   eta=eta,
                   save_interval=20*minute)

    if run:
        seq.run(display_hash=False,print_time=True)
        pn.save(sfname,seq) 

    
    return sfname
    
    


## This for all sims

In [ ]:
def to_named_tuple(params_list):
    from collections import namedtuple
    keys=list(params_list[0].keys())
    keys+=['count']
    params=namedtuple('params',keys)
    
    tuples_list=[]
    for count,p in enumerate(params_list):
        p2=params(count=count,
                  **p)
        tuples_list.append(p2)
        
        
    return tuples_list

In [ ]:
def make_do_params(all_params,verbose=False):
    do_params=[]
    for p in all_params:
        if os.path.exists(p.sfname):
            print("Skipping %s...already exists" % p.sfname)
        else:
            do_params+=[p]

    if verbose:
        print("%d sims" % len(do_params))    
        if len(do_params)<=15:
            print(do_params)
        else:
            print(do_params[:5],"...",do_params[-5:])        
    return do_params

In [ ]:
number_of_neurons=20
eta=1e-6
number_of_processes=4

## Continuous Fix

In [ ]:
func=run_one_continuous_fix

noise_mat=linspace(0,1,11)

all_params=[]
for n,noise in enumerate(noise_mat):
    sfname=f'sims-2021-11-19/continuous fix {number_of_neurons} neurons noise {noise:.1f}.asdf'
    
    p=Struct()
    p.eta=eta
    p.number_of_neurons=number_of_neurons
    p.sfname=sfname
    
    p.noise=noise
    
    all_params+=[p]

all_params=to_named_tuple(all_params)  

In [ ]:
# %%time
# print(func.__name__)
# func(all_params[0],overwrite=True)

In [ ]:
### premake the images
for params in tqdm(all_params):
    result=func.remote(params,run=False,overwrite=True)
    sfname=ray.get(result)
    print(sfname)

  0%|          | 0/11 [00:00<?, ?it/s]

(pid=92478) Splikes version  0.1.4
(pid=92478) Plasticnet version  0.1.12
sims-2021-11-19/continuous fix 20 neurons noise 0.0.asdf
sims-2021-11-19/continuous fix 20 neurons noise 0.1.asdf
sims-2021-11-19/continuous fix 20 neurons noise 0.2.asdf
sims-2021-11-19/continuous fix 20 neurons noise 0.3.asdf
sims-2021-11-19/continuous fix 20 neurons noise 0.4.asdf
sims-2021-11-19/continuous fix 20 neurons noise 0.5.asdf
sims-2021-11-19/continuous fix 20 neurons noise 0.6.asdf
sims-2021-11-19/continuous fix 20 neurons noise 0.7.asdf
sims-2021-11-19/continuous fix 20 neurons noise 0.8.asdf
sims-2021-11-19/continuous fix 20 neurons noise 0.9.asdf
sims-2021-11-19/continuous fix 20 neurons noise 1.0.asdf


In [ ]:
do_params=make_do_params(all_params,verbose=True)

11 sims
[params(eta=1e-06, number_of_neurons=20, sfname='sims-2021-11-19/continuous fix 20 neurons noise 0.0.asdf', noise=0.0, count=0), params(eta=1e-06, number_of_neurons=20, sfname='sims-2021-11-19/continuous fix 20 neurons noise 0.1.asdf', noise=0.1, count=1), params(eta=1e-06, number_of_neurons=20, sfname='sims-2021-11-19/continuous fix 20 neurons noise 0.2.asdf', noise=0.2, count=2), params(eta=1e-06, number_of_neurons=20, sfname='sims-2021-11-19/continuous fix 20 neurons noise 0.3.asdf', noise=0.30000000000000004, count=3), params(eta=1e-06, number_of_neurons=20, sfname='sims-2021-11-19/continuous fix 20 neurons noise 0.4.asdf', noise=0.4, count=4), params(eta=1e-06, number_of_neurons=20, sfname='sims-2021-11-19/continuous fix 20 neurons noise 0.5.asdf', noise=0.5, count=5), params(eta=1e-06, number_of_neurons=20, sfname='sims-2021-11-19/continuous fix 20 neurons noise 0.6.asdf', noise=0.6000000000000001, count=6), params(eta=1e-06, number_of_neurons=20, sfname='sims-2021-11-19/

In [ ]:
real_time=2*minute+ 38
print(time2str(real_time*len(do_params)/number_of_processes))

7 m, 14.50 s


In [ ]:
results = [func.remote(p) for p in do_params]
sfnames=ray.get(results)

(run_one_continuous_fix pid=92478) [
(pid=92477) Splikes version  0.1.4
(pid=92477) Plasticnet version  0.1.12
(pid=92477) macOS-14.4.1-arm64-arm-64bit
(pid=92477) Base Image File: asdf/bbsk081604_all_log2dog.asdf
(run_one_continuous_fix pid=92479) Grating time elapsed 8.76 s
(run_one_continuous_fix pid=92479) .
(run_one_continuous_fix pid=92482) [ [repeated 7x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
(pid=92482) Splikes version  0.1.4 [repeated 6x across cluster]
(pid=92482) Plasticnet version  0.1.12 [repeated 6x across cluster]
(pid=92482) macOS-14.4.1-arm64-arm-64bit [repeated 6x across cluster]
(pid=92482) Base Image File: asdf/bbsk081604_all_log2dog.asdf [repeated 6x across cluster]
(run_one_continuous_fix pid=92479) Grating time elapsed 13.17 s [repeated 8x across cluster]
(run_one_continuous_f

## Patch

In [ ]:
func=func=run_one_continuous_patch

closed_eye_noise_mat=linspace(0,1,21)

all_params=[]
for n,noise in enumerate(closed_eye_noise_mat):
    sfname=f'sims-2021-11-19/continuous patch {number_of_neurons} neurons noise {noise:.1f}.asdf'
    
    p=Struct()
    p.eta=eta
    p.number_of_neurons=number_of_neurons
    p.sfname=sfname
    
    p.noise=noise
    
    all_params+=[p]

all_params=to_named_tuple(all_params)  


In [ ]:
# %%time
# print(func.__name__)
# func(all_params[0],overwrite=True)

In [ ]:
do_params=make_do_params(all_params,verbose=True)

21 sims
[params(eta=1e-06, number_of_neurons=20, sfname='sims-2021-11-19/continuous patch 20 neurons noise 0.0.asdf', noise=0.0, count=0), params(eta=1e-06, number_of_neurons=20, sfname='sims-2021-11-19/continuous patch 20 neurons noise 0.1.asdf', noise=0.05, count=1), params(eta=1e-06, number_of_neurons=20, sfname='sims-2021-11-19/continuous patch 20 neurons noise 0.1.asdf', noise=0.1, count=2), params(eta=1e-06, number_of_neurons=20, sfname='sims-2021-11-19/continuous patch 20 neurons noise 0.2.asdf', noise=0.15000000000000002, count=3), params(eta=1e-06, number_of_neurons=20, sfname='sims-2021-11-19/continuous patch 20 neurons noise 0.2.asdf', noise=0.2, count=4)] ... [params(eta=1e-06, number_of_neurons=20, sfname='sims-2021-11-19/continuous patch 20 neurons noise 0.8.asdf', noise=0.8, count=16), params(eta=1e-06, number_of_neurons=20, sfname='sims-2021-11-19/continuous patch 20 neurons noise 0.9.asdf', noise=0.8500000000000001, count=17), params(eta=1e-06, number_of_neurons=20, sf

In [ ]:
real_time=8*minute+ 46
print(time2str(real_time*len(do_params)/number_of_processes))

46 m, 1.50 s


In [ ]:
results = [func.remote(p) for p in do_params]
sfnames=ray.get(results)

(pid=16830) Splikes version  0.1.4
(pid=16830) Plasticnet version  0.1.12
(pid=16830) macOS-14.4.1-arm64-arm-64bit
(pid=16830) Base Image File: asdf/bbsk081604_all_log2dog.asdf
(run_one_continuous_patch pid=16830) [
(run_one_continuous_patch pid=16830) Grating time elapsed 8.78 s
(run_one_continuous_patch pid=16830) .
(pid=16829) Splikes version  0.1.4 [repeated 3x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
(pid=16829) Plasticnet version  0.1.12 [repeated 3x across cluster]
(pid=16829) macOS-14.4.1-arm64-arm-64bit [repeated 3x across cluster]
(pid=16829) Base Image File: asdf/bbsk081604_all_log2dog.asdf [repeated 3x across cluster]
(run_one_continuous_patch pid=16829) [ [repeated 3x across cluster]
(run_one_continuous_patch pid=16829) .] Sequence Time Elapsed...2 m, 15.12 s
(run_one_continuous_patch pid

## Atropine

In [ ]:
func=func=run_one_continuous_blur


atropine_blur_mat=linspace(0,6,21)
closed_eye_noise_mat=linspace(0,1,11)

all_params=[]
for b,blur in enumerate(atropine_blur_mat):
    for n,noise in enumerate(closed_eye_noise_mat):
        sfname=f'sims-2021-11-19/continuous atropine {number_of_neurons} neurons noise {noise:.1f} blur {blur:0.1f}.asdf'

        p=Struct()
        p.eta=eta
        p.number_of_neurons=number_of_neurons
        p.sfname=sfname

        p.noise=noise
        p.blur=blur

        all_params+=[p]

all_params=to_named_tuple(all_params)  

In [ ]:
# %%time
# print(func.__name__)
# func(all_params[0],overwrite=True)

In [ ]:
### premake the images
for params in tqdm(all_params):
    result=func.remote(params,run=False,overwrite=True)
    sfname=ray.get(result)

  0%|          | 0/231 [00:00<?, ?it/s]

(run_one_continuous_blur pid=92479) Resolution float64 -> uint16
(run_one_continuous_blur pid=92479) Resolution float64 -> uint16
(run_one_continuous_blur pid=92479) Resolution float64 -> uint16
(run_one_continuous_blur pid=92479) Resolution float64 -> uint16
(run_one_continuous_blur pid=92479) Resolution float64 -> uint16
(run_one_continuous_blur pid=92479) Resolution float64 -> uint16
(run_one_continuous_blur pid=92479) Resolution float64 -> uint16
(run_one_continuous_blur pid=92479) Resolution float64 -> uint16
(run_one_continuous_blur pid=92479) Resolution float64 -> uint16
(run_one_continuous_blur pid=92479) Resolution float64 -> uint16
(run_one_continuous_blur pid=92479) Resolution float64 -> uint16
(run_one_continuous_blur pid=92479) Resolution float64 -> uint16
(run_one_continuous_blur pid=92479) Resolution float64 -> uint16
(run_one_continuous_blur pid=92479) Resolution float64 -> uint16
(run_one_continuous_blur pid=92479) Resolution float64 -> uint16
(run_one_continuous_blur 

In [ ]:
do_params=make_do_params(all_params,verbose=True)

231 sims
[params(eta=1e-06, number_of_neurons=20, sfname='sims-2021-11-19/continuous atropine 20 neurons noise 0.0 blur 0.0.asdf', noise=0.0, blur=0.0, count=0), params(eta=1e-06, number_of_neurons=20, sfname='sims-2021-11-19/continuous atropine 20 neurons noise 0.1 blur 0.0.asdf', noise=0.1, blur=0.0, count=1), params(eta=1e-06, number_of_neurons=20, sfname='sims-2021-11-19/continuous atropine 20 neurons noise 0.2 blur 0.0.asdf', noise=0.2, blur=0.0, count=2), params(eta=1e-06, number_of_neurons=20, sfname='sims-2021-11-19/continuous atropine 20 neurons noise 0.3 blur 0.0.asdf', noise=0.30000000000000004, blur=0.0, count=3), params(eta=1e-06, number_of_neurons=20, sfname='sims-2021-11-19/continuous atropine 20 neurons noise 0.4 blur 0.0.asdf', noise=0.4, blur=0.0, count=4)] ... [params(eta=1e-06, number_of_neurons=20, sfname='sims-2021-11-19/continuous atropine 20 neurons noise 0.6 blur 6.0.asdf', noise=0.6000000000000001, blur=6.0, count=226), params(eta=1e-06, number_of_neurons=20, 

In [ ]:
real_time=7*minute+ 37
print(time2str(real_time*len(do_params)/number_of_processes))

7 h, 19 m, 51.75 s


In [ ]:
results = [func.remote(p) for p in do_params]
sfnames=ray.get(results)

(run_one_continuous_blur pid=92478) [
(run_one_continuous_blur pid=92482) Grating time elapsed 8.43 s
(run_one_continuous_blur pid=92482) .
(run_one_continuous_blur pid=92482) [ [repeated 7x across cluster]
(run_one_continuous_blur pid=92482) .] Sequence Time Elapsed...2 m, 9.14 s
(run_one_continuous_blur pid=92478) Grating time elapsed 8.42 s [repeated 7x across cluster]
(run_one_continuous_blur pid=92478) . [repeated 7x across cluster]
(run_one_continuous_blur pid=92476) [
(run_one_continuous_blur pid=92478) .] Sequence Time Elapsed...2 m, 11.00 s [repeated 7x across cluster]
(run_one_continuous_blur pid=92482) Grating time elapsed 8.43 s
(run_one_continuous_blur pid=92478) [ [repeated 7x across cluster]
(run_one_continuous_blur pid=92476) .
(run_one_continuous_blur pid=92482) .] Sequence Time Elapsed...2 m, 9.19 s
(run_one_continuous_blur pid=92478) Grating time elapsed 8.41 s [repeated 7x across cluster]
(run_one_continuous_blur pid=92478) . [repeated 7x across cluster]
(run_one_co

## Contrast

In [ ]:
func=func=run_one_continuous_mask


contrast_mat=linspace(0,1,11)
mask_mat=array([0,1])
f_mat=array([10,30,50,70,90])


all_params=[]
for c,contrast in enumerate(contrast_mat):
    sfname=f'sims-2021-11-19/continuous contrast {number_of_neurons} neurons contrast {contrast:.1f}.asdf'

    p=Struct()
    p.eta=eta
    p.number_of_neurons=number_of_neurons
    p.sfname=sfname

    p.contrast=contrast
    p.mask=0
    p.f=10. # not used when mask=0

    all_params+=[p]

all_params=to_named_tuple(all_params)  

In [ ]:
do_params=make_do_params(all_params,verbose=True)

11 sims
[params(eta=1e-06, number_of_neurons=20, sfname='sims-2021-11-19/continuous contrast 20 neurons contrast 0.0.asdf', contrast=0.0, mask=0, f=10.0, count=0), params(eta=1e-06, number_of_neurons=20, sfname='sims-2021-11-19/continuous contrast 20 neurons contrast 0.1.asdf', contrast=0.1, mask=0, f=10.0, count=1), params(eta=1e-06, number_of_neurons=20, sfname='sims-2021-11-19/continuous contrast 20 neurons contrast 0.2.asdf', contrast=0.2, mask=0, f=10.0, count=2), params(eta=1e-06, number_of_neurons=20, sfname='sims-2021-11-19/continuous contrast 20 neurons contrast 0.3.asdf', contrast=0.30000000000000004, mask=0, f=10.0, count=3), params(eta=1e-06, number_of_neurons=20, sfname='sims-2021-11-19/continuous contrast 20 neurons contrast 0.4.asdf', contrast=0.4, mask=0, f=10.0, count=4), params(eta=1e-06, number_of_neurons=20, sfname='sims-2021-11-19/continuous contrast 20 neurons contrast 0.5.asdf', contrast=0.5, mask=0, f=10.0, count=5), params(eta=1e-06, number_of_neurons=20, sfnam

In [ ]:
# %%time
# print(func.__name__)
# func(all_params[0],overwrite=True)

In [ ]:
real_time=7*minute+ 55
print(time2str(real_time*len(do_params)/number_of_processes))

21 m, 46.25 s


In [ ]:
results = [func.remote(p) for p in do_params]
sfnames=ray.get(results)

(run_one_continuous_mask pid=92478) [
(run_one_continuous_mask pid=92480) Grating time elapsed 8.37 s
(run_one_continuous_mask pid=92480) .
(run_one_continuous_mask pid=92482) [ [repeated 7x across cluster]
(run_one_continuous_mask pid=92480) .] Sequence Time Elapsed...2 m, 9.18 s
(run_one_continuous_mask pid=92475) Grating time elapsed 8.38 s [repeated 7x across cluster]
(run_one_continuous_mask pid=92475) . [repeated 7x across cluster]
(run_one_continuous_mask pid=92480) [
(run_one_continuous_mask pid=92475) .] Sequence Time Elapsed...2 m, 10.66 s [repeated 7x across cluster]
(run_one_continuous_mask pid=92479) Grating time elapsed 8.35 s
(run_one_continuous_mask pid=92479) [ [repeated 2x across cluster]
(run_one_continuous_mask pid=92476) Grating time elapsed 8.35 s
(run_one_continuous_mask pid=92479) .
(run_one_continuous_mask pid=92479) .] Sequence Time Elapsed...2 m, 8.69 s
(run_one_continuous_mask pid=92480) Grating time elapsed 8.36 s
(run_one_continuous_mask pid=92480) . [repe

## Contrast with Mask

In [ ]:
func=func=run_one_continuous_mask


contrast_mat=linspace(0,1,11)
mask_mat=array([0,1])
f_mat=array([10,30,50,70,90])


all_params=[]
for c,contrast in enumerate(contrast_mat):
    for fi,f in enumerate(f_mat):
        sfname=f'sims-2021-11-19/continuous contrast {number_of_neurons} neurons contrast {contrast:.1f} mask f {f}.asdf'

        p=Struct()
        p.eta=eta
        p.number_of_neurons=number_of_neurons
        p.sfname=sfname

        p.contrast=contrast
        p.mask=1
        p.f=f # not used when mask=0

        all_params+=[p]

all_params=to_named_tuple(all_params)  


In [ ]:
# %%time
# print(func.__name__)
# func(all_params[0],overwrite=True)

In [ ]:
### premake the images
for params in tqdm(all_params):
    result=func.remote(params,run=False,overwrite=True)
    sfname=ray.get(result)

  0%|          | 0/55 [00:00<?, ?it/s]

(run_one_continuous_mask pid=92480) Mask ['bblais-masks-20210615/2021-06-15-D-A-fsig10.png', 'bblais-masks-20210615/2021-06-15-B-A-fsig10.png', 'bblais-masks-20210615/2021-06-15-D-A-fsig10.png', 'bblais-masks-20210615/2021-06-15-B-A-fsig10.png', 'bblais-masks-20210615/2021-06-15-A-A-fsig10.png', 'bblais-masks-20210615/2021-06-15-E-A-fsig10.png', 'bblais-masks-20210615/2021-06-15-A-A-fsig10.png', 'bblais-masks-20210615/2021-06-15-E-A-fsig10.png', 'bblais-masks-20210615/2021-06-15-E-A-fsig10.png', 'bblais-masks-20210615/2021-06-15-A-A-fsig10.png', 'bblais-masks-20210615/2021-06-15-E-A-fsig10.png', 'bblais-masks-20210615/2021-06-15-A-A-fsig10.png', 'bblais-masks-20210615/2021-06-15-B-A-fsig10.png', 'bblais-masks-20210615/2021-06-15-D-A-fsig10.png', 'bblais-masks-20210615/2021-06-15-C-A-fsig10.png', 'bblais-masks-20210615/2021-06-15-A-A-fsig10.png', 'bblais-masks-20210615/2021-06-15-D-A-fsig10.png', 'bblais-masks-20210615/2021-06-15-D-A-fsig10.png', 'bblais-masks-20210615/2021-06-15-D-A-fs

In [ ]:
do_params=make_do_params(all_params,verbose=True)

55 sims
[params(eta=1e-06, number_of_neurons=20, sfname='sims-2021-11-19/continuous contrast 20 neurons contrast 0.0 mask f 10.asdf', contrast=0.0, mask=1, f=10, count=0), params(eta=1e-06, number_of_neurons=20, sfname='sims-2021-11-19/continuous contrast 20 neurons contrast 0.0 mask f 30.asdf', contrast=0.0, mask=1, f=30, count=1), params(eta=1e-06, number_of_neurons=20, sfname='sims-2021-11-19/continuous contrast 20 neurons contrast 0.0 mask f 50.asdf', contrast=0.0, mask=1, f=50, count=2), params(eta=1e-06, number_of_neurons=20, sfname='sims-2021-11-19/continuous contrast 20 neurons contrast 0.0 mask f 70.asdf', contrast=0.0, mask=1, f=70, count=3), params(eta=1e-06, number_of_neurons=20, sfname='sims-2021-11-19/continuous contrast 20 neurons contrast 0.0 mask f 90.asdf', contrast=0.0, mask=1, f=90, count=4)] ... [params(eta=1e-06, number_of_neurons=20, sfname='sims-2021-11-19/continuous contrast 20 neurons contrast 1.0 mask f 10.asdf', contrast=1.0, mask=1, f=10, count=50), params(

In [ ]:
real_time=8*minute+ 8
print(time2str(real_time*len(do_params)/number_of_processes))

1 h, 51 m, 50.00 s


In [ ]:
results = [func.remote(p) for p in do_params]
sfnames=ray.get(results)

(run_one_continuous_mask pid=92478) [
(run_one_continuous_mask pid=92475) Grating time elapsed 8.39 s
(run_one_continuous_mask pid=92475) .
(run_one_continuous_mask pid=92482) [ [repeated 7x across cluster]
(run_one_continuous_mask pid=92479) .] Sequence Time Elapsed...2 m, 10.77 s
(run_one_continuous_mask pid=92480) Grating time elapsed 8.38 s [repeated 7x across cluster]
(run_one_continuous_mask pid=92480) . [repeated 7x across cluster]
(run_one_continuous_mask pid=92479) [
(run_one_continuous_mask pid=92480) .] Sequence Time Elapsed...2 m, 12.01 s [repeated 7x across cluster]
(run_one_continuous_mask pid=92479) Grating time elapsed 8.35 s
(run_one_continuous_mask pid=92480) [ [repeated 7x across cluster]
(run_one_continuous_mask pid=92479) .
(run_one_continuous_mask pid=92475) Grating time elapsed 8.38 s
(run_one_continuous_mask pid=92479) .] Sequence Time Elapsed...2 m, 10.19 s
(run_one_continuous_mask pid=92478) . [repeated 7x across cluster]
(run_one_continuous_mask pid=92478) Gr